# Data preparation – expenditure

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, @guerrero_oa)

This tutorial will show how to prepared a dataset containing expenditure programmes that are linked, somehow, to the development indicators. I will assume that the raw expenditure data has already certain structure, and I will provide examples of structures with different levels of granularity. The aim will be to prepare two files: (1) a disbursement schedule and (2) a relational table.

## Import the necessary python libraries to manipulate data

In [1]:
import pandas as pd
import numpy as np

## On expenditure linked data

In any impact evaluation of public expenditure, it is necessary to have some information about the level of expenditure that is devoted to an indicator. Usually, broad tranches such as educatino, public health, or national defense are used for that purpose. In the context of multidimensional impact evaluation, these data need to be more disaggregated. In an ideal scenario, there should be one development indicator directly linked to one expenditure programme. In real life, such one-to-one mapping does not exist because there can be multiple government prgorammes designed to affect the same indicator, or several indicators affected by the same programme. PPI was designed with this imperfect matching in mind.

Today, it is still difficult to find expenditure datasets with a high degree of disaggregation. Thus, in this tutorial, I will show two examples. Before elaborating in these examples, I need to explain how PPI uses the expenditure data and some important considerations that one need to take into account before preparing the final dataset.


## PII and expenditure data

To accomodate different linkage qualities between expenditure and indicators, PPI relies on a model of how the government priorises it spending (see more in the Model chapter of the book). For instance, if there is only aggregate data for the tranche of education, but there are several indicators capturing different policy issues within education, the model determines the spending distribution within education endogenously. However, if the used has data on how the educational budget was actually allocated across more fine-grained policy issues, this information can be incorporated into PPI to rely less on the model and more on the data.

## Temporal factors

Before preparing the example datasets, it is important to mention three important adjustments that should be done beforehand:

* Accounting for inflation
* Accounting for population growth
* Accounting for spending inertia

Controlling for these (and perhaps other) temporal variables is important to remove their influence from the expenditure-indicator relationship that PPI models. Dealing for inflacion is straightforward as it consists of turning the expenditure time series in constant monetary units. Population growth is also easy as one needs to divide the data by the populaiton size (which changes through time) to obtain per capita spending.

Once the two previous adjustments have been made, there may still be certain inertia in the spending time series. This should also be removed for technical reasons regarding the model in PPI. In a nutshell, this is necessary because of the calibration of a parameter $\beta_i$ that normalises the expenditure destined to indicator $i$ into 0 and 1 to determine the probability of success of the indicator.

Note that $\beta_i$ does not have a time sub-index, which means that it is a cnostant parameter. This means that, if the expenditure data related to $i$ has a positive trend, the latter periods of the simulation will tend to have a higher probability of success than the early ones. These inter-temporal differenes in success rates is an artifact of not removing the trend component of expenditure, as the indicator data does not suggest a systematic improvement in success rates, but rather that spending in policy issues becomes more expensive–in real terms–with time.

There are various ways in which one could remove the trend component form a, expenditure time series. In the book, we use the naive approach of simply calculating the inter-temporal average of each expenditure programme, and applying it in every period. This is a simple approach that meets the technical level of the book, and that is acceptable is one is not concerned about specific points in time during the sample period. However, one may want to consider more nuanced methods like running a linear regression and taking the differences with respect to the predicted values, or a Hodrick–Prescott filter that is popular in macroecnomics, or any signal-processing method (as we do in http://dx.doi.org/10.2139/ssrn.4101378).

In this tutorial, I assume that the raw expenditure data consists of a table with expenditure time series of various expenditure SDGs; and that these data has already been prepared to account for the afoerementioned factors. Hence, in absence of expenditure programmes, the SDGs provide the imperfect link between spending data and indicators.

## Load expenditure data

In [2]:
data_exp = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/raw_data/raw_expenditure.csv')
data_indi = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_indicators.csv')

In [3]:
data_exp

sdg       2000       2001       2002       2003       2004       2005  \
0     1  76.101460  75.407460  78.815556  75.029466  77.059208  79.300657   
1     2  27.733506  30.359304  27.047538  29.349506  27.634046  26.086381   
2     3  14.967628  16.289916  14.319872  15.779269  14.013817  15.585717   
3     4  53.130442  53.308972  52.664089  50.485033  52.478833  50.363236   
4     5  34.878440  34.487904  32.202287  34.660567  34.719991  35.556516   
5     6  18.067375  18.913180  22.571486  18.391029  22.621347  19.003758   
6     7  72.001893  75.556812  76.236047  72.545503  72.029558  73.634223   
7     8  83.155957  84.478164  86.623278  83.927198  82.249885  86.156839   
8     9  64.542332  63.104612  66.126611  66.363337  66.795704  66.385627   
9    10  29.036009  30.935416  31.815671  31.601834  31.240002  29.055714   
10   11  98.579207  94.406885  94.759591  94.354614  94.438297  96.205712   
11   12  26.908762  25.811333  28.885238  27.158590  26.021063  26.900091   
12   13  67.510147  67.636903  64.391516  66.977226  63.530965  67.676177   
13   14  41.705704  42.162651  39.641780  43.790009  42.684484  42.092191   
14   15  74.046534  71.124408  72.838837  71.536961  74.949860  71.628875   
15   16  61.380688  59.338317  60.504732  61.493368  61.180170  59.862393   
16   17  78.658195  78.196355  76.788063  74.802415  76.531776  74.214084   

         2006       2007       2008  ...       2013       2014       2015  \
0   77.349554  75.193281  78.546466  ...  79.629446  77.001899  79.495359   
1   30.635200  26.983303  30.899117  ...  30.855951  28.324059  29.323354   
2   15.559116  15.749640  17.141196  ...  18.282329  17.634532  18.607787   
3   52.765069  54.764521  52.262026  ...  54.379266  52.031701  54.981681   
4   35.509773  34.841871  35.996053  ...  34.162981  34.716396  35.683038   
5   19.343980  22.965608  20.894071  ...  18.347396  19.472177  19.697528   
6   74.154735  74.416242  75.117645  ...  74.409750  76.023278  73.086175   
7   84.723291  83.249383  83.727024  ...  85.565120  85.056890  84.153154   
8   65.281954  65.939985  66.478073  ...  63.967526  62.304570  65.869197   
9   31.643980  33.789675  31.471496  ...  31.432146  32.903993  33.036646   
10  96.079194  96.778109  96.195213  ...  98.236943  95.708395  94.799610   
11  27.462561  27.987761  25.528759  ...  27.323101  29.466255  25.702529   
12  63.581576  66.115134  67.954801  ...  63.055567  65.259805  67.357876   
13  39.951496  39.054187  40.781689  ...  43.378198  39.517822  39.859026   
14  73.923696  70.547401  72.571241  ...  71.171402  73.593802  73.498641   
15  62.348168  60.684170  62.251058  ...  60.129870  60.920475  62.615021   
16  74.234622  76.466451  78.390283  ...  74.100442  74.691934  75.014447   

         2016       2017       2018       2019       2020       2021  \
0   75.265310  79.182231  76.737024  75.045972  76.856820  79.631567   
1   29.304476  28.076520  30.885578  27.761037  30.363916  27.955566   
2   18.142629  16.176429  18.696378  15.949757  15.764068  14.882900   
3   50.097032  54.905479  52.281982  52.357182  51.329552  50.403684   
4   34.874965  34.199573  34.333913  35.516865  34.550722  33.453961   
5   22.725358  18.776422  21.232735  18.807887  21.389369  21.560489   
6   75.281867  72.221927  75.506844  72.508493  73.497118  74.220234   
7   86.272066  82.022157  83.230945  83.181246  84.613776  84.635270   
8   64.200016  62.695402  62.178968  62.412289  62.851275  64.514739   
9   30.189648  32.627706  31.378979  30.142284  29.106578  29.439161   
10  97.190876  98.172784  97.663151  94.650243  95.057279  95.503399   
11  25.860361  28.955940  28.000066  27.912614  26.494042  26.306015   
12  63.420815  66.100661  66.905984  64.091037  65.333765  67.290226   
13  43.482612  42.335610  42.116567  40.250013  42.695480  39.033188   
14  71.399244  70.128430  72.031360  74.537056  72.986144  73.673080   
15  61.175588  60.655816  60.192481  62.343959  59.879520  59.626265   
16  76

First, let us check which SDGs are covered in both the expenditure and the indicators datasets. Then, we need to make sure that every SDG has at least one instrumental indicator.

In [4]:
data_exp = data_exp[data_exp.sdg.isin(data_indi.sdg.values)]
data_exp = data_exp[data_exp.sdg.isin(data_indi[data_indi.instrumental==1].sdg.values)]

In [5]:
data_exp

sdg       2000       2001       2002       2003       2004       2005  \
0     1  76.101460  75.407460  78.815556  75.029466  77.059208  79.300657   
1     2  27.733506  30.359304  27.047538  29.349506  27.634046  26.086381   
2     3  14.967628  16.289916  14.319872  15.779269  14.013817  15.585717   
3     4  53.130442  53.308972  52.664089  50.485033  52.478833  50.363236   
4     5  34.878440  34.487904  32.202287  34.660567  34.719991  35.556516   
5     6  18.067375  18.913180  22.571486  18.391029  22.621347  19.003758   
6     7  72.001893  75.556812  76.236047  72.545503  72.029558  73.634223   
7     8  83.155957  84.478164  86.623278  83.927198  82.249885  86.156839   
8     9  64.542332  63.104612  66.126611  66.363337  66.795704  66.385627   
10   11  98.579207  94.406885  94.759591  94.354614  94.438297  96.205712   
11   12  26.908762  25.811333  28.885238  27.158590  26.021063  26.900091   
12   13  67.510147  67.636903  64.391516  66.977226  63.530965  67.676177   
13   14  41.705704  42.162651  39.641780  43.790009  42.684484  42.092191   
14   15  74.046534  71.124408  72.838837  71.536961  74.949860  71.628875   
15   16  61.380688  59.338317  60.504732  61.493368  61.180170  59.862393   
16   17  78.658195  78.196355  76.788063  74.802415  76.531776  74.214084   

         2006       2007       2008  ...       2013       2014       2015  \
0   77.349554  75.193281  78.546466  ...  79.629446  77.001899  79.495359   
1   30.635200  26.983303  30.899117  ...  30.855951  28.324059  29.323354   
2   15.559116  15.749640  17.141196  ...  18.282329  17.634532  18.607787   
3   52.765069  54.764521  52.262026  ...  54.379266  52.031701  54.981681   
4   35.509773  34.841871  35.996053  ...  34.162981  34.716396  35.683038   
5   19.343980  22.965608  20.894071  ...  18.347396  19.472177  19.697528   
6   74.154735  74.416242  75.117645  ...  74.409750  76.023278  73.086175   
7   84.723291  83.249383  83.727024  ...  85.565120  85.056890  84.153154   
8   65.281954  65.939985  66.478073  ...  63.967526  62.304570  65.869197   
10  96.079194  96.778109  96.195213  ...  98.236943  95.708395  94.799610   
11  27.462561  27.987761  25.528759  ...  27.323101  29.466255  25.702529   
12  63.581576  66.115134  67.954801  ...  63.055567  65.259805  67.357876   
13  39.951496  39.054187  40.781689  ...  43.378198  39.517822  39.859026   
14  73.923696  70.547401  72.571241  ...  71.171402  73.593802  73.498641   
15  62.348168  60.684170  62.251058  ...  60.129870  60.920475  62.615021   
16  74.234622  76.466451  78.390283  ...  74.100442  74.691934  75.014447   

         2016       2017       2018       2019       2020       2021  \
0   75.265310  79.182231  76.737024  75.045972  76.856820  79.631567   
1   29.304476  28.076520  30.885578  27.761037  30.363916  27.955566   
2   18.142629  16.176429  18.696378  15.949757  15.764068  14.882900   
3   50.097032  54.905479  52.281982  52.357182  51.329552  50.403684   
4   34.874965  34.199573  34.333913  35.516865  34.550722  33.453961   
5   22.725358  18.776422  21.232735  18.807887  21.389369  21.560489   
6   75.281867  72.221927  75.506844  72.508493  73.497118  74.220234   
7   86.272066  82.022157  83.230945  83.181246  84.613776  84.635270   
8   64.200016  62.695402  62.178968  62.412289  62.851275  64.514739   
10  97.190876  98.172784  97.663151  94.650243  95.057279  95.503399   
11  25.860361  28.955940  28.000066  27.912614  26.494042  26.306015   
12  63.420815  66.100661  66.905984  64.091037  65.333765  67.290226   
13  43.482612  42.335610  42.116567  40.250013  42.695480  39.033188   
14  71.399244  70.128430  72.031360  74.537056  72.986144  73.673080   
15  61.175588  60.655816  60.192481  62.343959  59.879520  59.626265   
16  76.927439  77.064367  74.237434  78.963031  78.070441  76.614831   

         2022  
0   75.978326  
1   29.489619  
2   16.209015  
3   52.877299  
4   35.459352  
5   21.960413  
6   76.228077  
7   83.661153  
8   66.459689  

## Assembling disbursement schedule

The following stem is critical. In PPI, each simulation runs for $T$ steps. Each of these simulation steps <strong>does not</strong> correspond to a calendar period. Thus, the user needs to convert the expenditure data into a format that matches the number of simulation steps. Since most indicator and expenditure data are relatively short, $T$ tends to be larger than the sample period. Therefore, the expenditure series need to be "streched" to provide the government agent with a disbursement schedule that determines the level of spending in each simulation step.

For these tutorials, let us use $T=69$, so that three simulation steps are equivalent to one calendar year (because there are 23 years in the data). The Calibration chapter of the book explains that the choice of $T$ should be at least 40 to give enough time for agents to learn. Values beyond 50 do not change the results in a significant way, but rather impose a computational burden. Thus, it is recommended to establish a $T$ between 40 and 200 that is easy to translate into calendar time.

To adjust the data to $T$, I will assume that, within each year, the government spends the same amount. Effectively, this means that one only needs to divide each annual observation by 3 and repeat it three times to elongate the time series. Of course, one does not need to assume an even temporal split. if nuanced information about how the government spends its resoures is available, it is easy to intruced a more detaiuled disbursement schedule.

In [6]:
years = [column_name for column_name in data_exp.columns if str(column_name).isnumeric()]
periods = len(years)
T = 69
t = int(T/periods)

new_rows = []
for index, row in data_exp.iterrows():
    new_row = [row.sdg]
    for year in years:
        new_row += [int(row[year]) for i in range(t)]
    new_rows.append(new_row)
    
df_exp = pd.DataFrame(new_rows, columns=['sdg']+list(range(T)))

In [7]:
df_exp

sdg   0   1   2   3   4   5   6   7   8  ...  59  60  61  62  63  64  65  \
0    1.0  76  76  76  75  75  75  78  78  78  ...  75  76  76  76  79  79  79   
1    2.0  27  27  27  30  30  30  27  27  27  ...  27  30  30  30  27  27  27   
2    3.0  14  14  14  16  16  16  14  14  14  ...  15  15  15  15  14  14  14   
3    4.0  53  53  53  53  53  53  52  52  52  ...  52  51  51  51  50  50  50   
4    5.0  34  34  34  34  34  34  32  32  32  ...  35  34  34  34  33  33  33   
5    6.0  18  18  18  18  18  18  22  22  22  ...  18  21  21  21  21  21  21   
6    7.0  72  72  72  75  75  75  76  76  76  ...  72  73  73  73  74  74  74   
7    8.0  83  83  83  84  84  84  86  86  86  ...  83  84  84  84  84  84  84   
8    9.0  64  64  64  63  63  63  66  66  66  ...  62  62  62  62  64  64  64   
9   11.0  98  98  98  94  94  94  94  94  94  ...  94  95  95  95  95  95  95   
10  12.0  26  26  26  25  25  25  28  28  28  ...  27  26  26  26  26  26  26   
11  13.0  67  67  67  67  67  67  64  64  64  ...  64  65  65  65  67  67  67   
12  14.0  41  41  41  42  42  42  39  39  39  ...  40  42  42  42  39  39  39   
13  15.0  74  74  74  71  71  71  72  72  72  ...  74  72  72  72  73  73  73   
14  16.0  61  61  61  59  59  59  60  60  60  ...  62  59  59  59  59  59  59   
15  17.0  78  78  78  78  78  78  76  76  76  ...  78  78  78  78  76  76  76   

    66  67  68  
0   75  75  75  
1   29  29  29  
2   16  16  16  
3   52  52  52  
4   35  35  35  
5   21  21  21  
6   76  76  76  
7   83  83  83  
8   66  66  66  
9   94  94  94  
10  25  25  25  
11  67  67  67  
12  40  40  40  
13  72  72  72  
14  59  59  59  
15  78  78  78  

[16 rows x 70 columns]

## The relational table

The last part of this tutorial consists of building a dataset linking each expenditure program to the indicators that it is supposed to affect. This dataset consists of a table with the unique ids of the indicators in the first column, and the ids of the programmes that are designed to affect them go along the rest of the columns (as many columns as needed) in the same row of the programme. Recall that, in this tutorial the SDGs as as programmes. In addition, you need to make sure that: 

* All instrumental indicators are reported in this table
* All programmes (SDGs) that appear in the disbursement schedule also appear in this table
* All the indicators reported in this table are only instrumental

Failure to comply with any of these requirements will make PPI to throw an error.

In [25]:
is_instrumental = dict(zip(data_indi.seriesCode, data_indi.instrumental==1))

rel_dict = dict([(code, []) for code in data_indi.seriesCode if is_instrumental[code]])
for index, row in data_indi.iterrows():
    if row.seriesCode in rel_dict:
        rel_dict[row.seriesCode].append(row.sdg)
    
n_cols = max([len(value) for value in rel_dict.values()])

M = [['' for i in range(n_cols+1)] for code in rel_dict.values()]
for i, items in enumerate(rel_dict.items()):
    sdg, indis = items
    M[i][0] = sdg
    for j, indi in enumerate(indis):
        M[i][j+1] = indi

df_rel = pd.DataFrame(M, columns=['seriesCode']+list(range(n_cols)))

In [26]:
df_rel

seriesCode   0
0    sdg5_familypl   5
1      sdg11_slums  11
2         sdg1_wpc   1
3      sdg1_320pov   1
4    sdg2_undernsh   2
..             ...  ..
66       sdg16_rsf  16
67   sdg16_justice  16
68     sdg17_govex  17
69    sdg17_govrev  17
70  sdg17_statperf  17

[71 rows x 2 columns]

## Save data

In [27]:
df_exp.to_csv('clean_data/data_expenditure.csv', index=False)
df_rel.to_csv('clean_data/data_relational_table.csv', index=False)